In [1]:
# Extract error types from: "conll14st-preprocessed.conll.ann" file
# Separate each word of a particular error type in its own list.
l1=[]
l4=[]
l5=[]
l6=[]
l7=[]
l11=[]
l14=[]
def extract_err_type(colnn_ann):
    f = open(colnn_ann, 'r')
    for line in f:
        parts = line.split()
        if(len(parts)>0):
            if(line[0:6]=="<TYPE>"):
                type_ = line[6:len(parts[0])-7]
            elif(line[0:12]=="<CORRECTION>"):
                corrected_word = line[12:len(line)-14] 
                if type_ =="Vt":
                    l1.append(line[12:len(line)-14])   
                if type_ =="SVA":
                    l5.append(line[12:len(line)-14])
                if type_ =="ArtOrDet":
                    l6.append(line[12:len(line)-14])
                if type_ =="Prep":
                    l11.append(line[12:len(line)-14])
    return "Done"

In [2]:
# Pass in to the function, the path to conll14st-preprocessed.conll.ann in your system
extract_err_type("/Users/highsierra/Tech-Skills/Labortory/ML-Fundamentals/error_types_conll14/conll14st-preprocessed.conll.ann")

'Done'

In [3]:
# Create a corresponding list containing the error type of each word
# Map between each word and its error type
ll1=[]
for i in range(len(l1)):
    ll1.append('Vt')

ll5=[]
for i in range(len(l5)):
    ll5.append('SVA')

ll6=[]
for i in range(len(l6)):
    ll6.append('ArtOrDet')

ll11=[]
for i in range(len(l11)):
    ll11.append('Prep')


In [4]:
# Separate the words (input) from the error types (target)
import itertools

targets = list(itertools.chain(ll1, ll5, ll6, ll11))
words = list(itertools.chain(l1, l5, l6, l11))

In [5]:
import re


with open("/Users/highsierra/conll14st-preprocessed.m2") as f:
    lines = f.read()
tests = lines.split("\n\n")

documents = []
doc = []

for sen in range(0, len(tests)):
    if re.search("Vt", str(tests[sen])) or re.search("SVA", str(tests[sen])) or re.search("ArtOrDet", str(tests[sen])) or re.search("Prep", str(tests[sen])):
        documents.append(str(tests[sen]))
        doc += str(tests[sen]).split(".\n")

cln = []
for k in range(0, len(doc)):
    if doc[k][0] == 'S':
        cln.append(doc[k])

flip = []
for sen in range(0, len(cln)):
    # Remove all the special characters
    err = re.sub(r"[^a-zA-Z/'/]", ' ', str(cln[sen]))

    # Substituting multiple spaces with single space
    err = re.sub(r'\s+', ' ', err, flags=re.I)

    # Removing prefixed 'S' and ending with 'n'
    err = re.sub(r"^S", '', err)
    err = re.sub(r"\.\n", '', err)

    # Removing trailling space
    err = re.sub(r'^\s', '', err, flags=re.I)
    err = re.sub(r'\s$', '', err, flags=re.I)

    flip.append(err)
    #print(errSen)
print(len(flip))
 




10342


In [6]:
import spacy
import numpy as np

nlp = spacy.load("en_core_web_sm")
l1=[]
l2=[]

for text in flip:
    doc = nlp(text)
    l1 = list([token.text, token.pos_, token.tag_, token.dep_] for token in doc if token.text in words)
    #lst3 = [value for value in words if value in lst] 
    l2.extend(l1)
    arr = np.array(l2)

print(arr.shape)


(125743, 4)


In [7]:
import spacy
import numpy as np

nlp = spacy.load("en_core_web_sm")
l1=[]
l2=[]

for text in flip:
    doc = nlp(text)
    l1 = list([token.text, token.pos_, token.tag_, token.dep_] for token in doc)
    l2.extend(l1)
    arrFull = np.array(l2)

print(arrFull.shape)

(249092, 4)


In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


transformer = ColumnTransformer(transformers=[('cat', OneHotEncoder(), [0, 1, 2, 3])], remainder='passthrough')

arr = transformer.fit_transform(arr)
print(arr.toarray())
print(arr.toarray().shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(125743, 958)


In [10]:
print(len(words))

13772


In [11]:
# Encode the input using glove embeddings   

from zeugma.embeddings import EmbeddingTransformer

glove = EmbeddingTransformer('glove') 
X = glove.transform(words)

Using TensorFlow backend.


In [12]:
print(X)
print(X.shape)

[[ 0.00094602 -0.084355    0.19994    ...  0.27858     0.20168
  -0.71859   ]
 [ 0.057466    0.1789     -0.62778    ... -0.075557    0.24192
   0.002247  ]
 [-0.52309    -0.68026    -0.46072    ...  0.52221     0.70644
  -0.25232   ]
 ...
 [ 0.32543    -0.089637   -0.14733    ... -0.6175     -0.54592
  -0.069893  ]
 [ 0.32543    -0.089637   -0.14733    ... -0.6175     -0.54592
  -0.069893  ]
 [-0.32929    -0.16037     0.10785    ...  0.021972   -0.73638
   0.65277   ]]
(13772, 25)


In [13]:
print(arr.toarray().dtype)
print(np.sum(arr))

float64
502972.0


In [14]:
arr2d= np.reshape(arr, (2, -1))
print(arr2d.toarray())
print(arr2d.shape)
print(np.sum(arr2d)) 

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(2, 60230897)
502972.0


In [15]:
wall = np.zeros((2,120456798),dtype=np.float64)

In [16]:
print(wall)
print(np.sum(wall))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0.0


In [17]:
wall[:, 0:60230897] += arr2d

In [18]:
print(wall.shape)
print(np.sum(wall))

(2, 120456798)
502972.0


In [19]:
wall2d= np.reshape(wall, (13772,-1))

In [20]:
print(wall2d.shape)
print(np.sum(wall2d))

(13772, 17493)
502972.0


In [21]:
print("concatenate column wise")
concat = np.concatenate((X,wall2d),axis=1)
print(concat)

concatenate column wise
[[ 0.00094602 -0.084355    0.19994    ...  0.          0.
   0.        ]
 [ 0.057466    0.1789     -0.62778002 ...  0.          0.
   0.        ]
 [-0.52309    -0.68026    -0.46072    ...  0.          0.
   0.        ]
 ...
 [ 0.32543001 -0.089637   -0.14733    ...  0.          0.
   0.        ]
 [ 0.32543001 -0.089637   -0.14733    ...  0.          0.
   0.        ]
 [-0.32929    -0.16037001  0.10785    ...  0.          0.
   0.        ]]


In [22]:
print(concat.shape)

(13772, 17518)


In [23]:
# Encode the target vector in binary 

from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
ybin = lb.fit(targets)
lb.classes_

array(['ArtOrDet', 'Prep', 'SVA', 'Vt'], dtype='<U8')

In [24]:
y = lb.transform(targets)
print(y)
print(y.shape)

[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 ...
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]]
(13772, 4)


### Random Forrest with added features

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(concat, y, test_size=0.2, random_state=0)

In [26]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_features=20, n_estimators=200, random_state=0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=20,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [27]:
y_pred = classifier.predict(X_test)

In [29]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


print(classification_report(y_test,y_pred,target_names=lb.classes_))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

    ArtOrDet       0.86      0.96      0.91      1333
        Prep       0.97      0.68      0.80       525
         SVA       0.69      0.87      0.77       288
          Vt       0.99      0.13      0.23       609

   micro avg       0.85      0.71      0.78      2755
   macro avg       0.88      0.66      0.68      2755
weighted avg       0.89      0.71      0.72      2755
 samples avg       0.71      0.71      0.71      2755

0.7125226860254084


/opt/miniconda3/envs/tensorflow1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### MLP with 4 hidden layers & added features

In [30]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(300,200,200,100), max_iter=500, activation = 'relu', solver='adam', random_state=21, tol=0.000000001)
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300, 200, 200, 100), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=21, shuffle=True, solver='adam',
              tol=1e-09, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [31]:
y_pred = clf.predict(X_test)

In [32]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


print(classification_report(y_test,y_pred,target_names=lb.classes_))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

    ArtOrDet       0.81      0.93      0.87      1333
        Prep       0.98      0.68      0.80       525
         SVA       0.88      0.79      0.83       288
          Vt       0.80      0.82      0.81       609

   micro avg       0.84      0.84      0.84      2755
   macro avg       0.87      0.80      0.83      2755
weighted avg       0.85      0.84      0.84      2755
 samples avg       0.84      0.84      0.84      2755

0.8377495462794918


/opt/miniconda3/envs/tensorflow1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
print(y)
print(y.shape)

[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 ...
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]]
(13772, 4)


In [34]:
print(X)
print(X.shape)

[[ 0.00094602 -0.084355    0.19994    ...  0.27858     0.20168
  -0.71859   ]
 [ 0.057466    0.1789     -0.62778    ... -0.075557    0.24192
   0.002247  ]
 [-0.52309    -0.68026    -0.46072    ...  0.52221     0.70644
  -0.25232   ]
 ...
 [ 0.32543    -0.089637   -0.14733    ... -0.6175     -0.54592
  -0.069893  ]
 [ 0.32543    -0.089637   -0.14733    ... -0.6175     -0.54592
  -0.069893  ]
 [-0.32929    -0.16037     0.10785    ...  0.021972   -0.73638
   0.65277   ]]
(13772, 25)


### MLP with 4 hidden layers & No added features

In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [40]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(300,200,200,100), max_iter=500, activation = 'relu', solver='adam', random_state=21, tol=0.000000001)
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300, 200, 200, 100), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=21, shuffle=True, solver='adam',
              tol=1e-09, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [41]:
y_pred = clf.predict(X_test)

In [42]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


print(classification_report(y_test,y_pred,target_names=lb.classes_))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

    ArtOrDet       0.85      0.98      0.91      1333
        Prep       0.96      0.68      0.80       525
         SVA       0.61      0.58      0.60       288
          Vt       0.79      0.74      0.76       609

   micro avg       0.83      0.83      0.83      2755
   macro avg       0.81      0.75      0.77      2755
weighted avg       0.84      0.83      0.82      2755
 samples avg       0.83      0.83      0.83      2755

0.8275862068965517


/opt/miniconda3/envs/tensorflow1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### MLP with 3 hidden layers 

In [36]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(300,200,100), max_iter=500, activation = 'relu', solver='adam', random_state=21, tol=0.000000001)
clf.fit(X_train, y_train) 

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300, 200, 100), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=21, shuffle=True, solver='adam',
              tol=1e-09, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [37]:
y_pred = clf.predict(X_test)

In [38]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


print(classification_report(y_test,y_pred,target_names=lb.classes_))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

    ArtOrDet       0.86      0.98      0.92      1333
        Prep       0.97      0.68      0.80       525
         SVA       0.63      0.45      0.53       288
          Vt       0.79      0.74      0.76       609

   micro avg       0.84      0.82      0.83      2755
   macro avg       0.81      0.71      0.75      2755
weighted avg       0.84      0.82      0.82      2755
 samples avg       0.82      0.82      0.82      2755

0.8156079854809437


/opt/miniconda3/envs/tensorflow1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
